In [1]:
import pandas as pd
import duckdb
import numpy as np

176. Вторая по величине зарплата  
Напишите решение для поиска второй по величине уникальной заработной платы в Employeeтаблице. Если второй по величине заработной платы нет, верните значение  null (return None in Pandas).

In [2]:
data = [[1, 100], [2, 200], [3, 300]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,100
1,2,200
2,3,300


In [17]:
data = [[1, 0], [2, 0], [3, 1]]
employee = pd.DataFrame(data, columns=['id', 'salary']).astype({'id':'int64', 'salary':'int64'})
employee.head()

,id,salary
0,1,0
1,2,0
2,3,1


In [ ]:
# мое решение
employee = employee.drop_duplicates(subset=['salary']) 
employee['rank'] = employee['salary'].rank(method='dense', ascending=False).astype('int64')
data_1 = [1,2]
ranke_1 = pd.DataFrame(data_1, columns=['ranke_1']).astype({'ranke_1':'int64'})
union_table = ranke_1.merge(employee, left_on='ranke_1', right_on='rank', how='left')
union_table[union_table['ranke_1'] == 2][['salary']].rename(columns={'salary': 'SecondHighestSalary'})

In [ ]:
# решение GPT
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
result = (
    salaries.iloc[1:2]
    .to_frame(name='SecondHighestSalary')
)
if result.empty:
    result = pd.DataFrame({'SecondHighestSalary': [None]})

,SecondHighestSalary
0,0


In [19]:
query = """
with t1 as (
    select 
        salary,
        dense_rank() over(order by salary desc) as rank
    from employee
    )
select
    case when count(*) < 1 then null
         else max(salary) end as SecondHighestSalary
from t1
where rank > 1
"""
result = duckdb.query(query).to_df()
result

,SecondHighestSalary
0,0


177. N-я самая высокая зарплата  
Напишите решение для поиска самой высокой уникальной заработной платы в таблице. Если уникальных зарплат  меньше, чем указано , верните значение .nthEmployeennull

In [8]:
N = 2
col_name = f'getNthHighestSalary({N})'
salaries = employee['salary'].drop_duplicates().sort_values(ascending=False)
if N > 0 and len(salaries) >= N:
    result = salaries.iloc[N-1:N].to_frame(name=col_name)
else:
    result = pd.DataFrame({col_name: [None]})
result

,getNthHighestSalary(2)
1,200


In [ ]:
N = 2
query = """
with t1 as (
    select 
        e.salary,
        dense_rank() over(order by e.salary desc) as rank
    from employee e
    )
select
    case when count(*) < 1 or N <= 0 then null
        else max(t1.salary) end as SecondHighestSalary
from t1
where rank > N - 1
"""
result = duckdb.query(query).to_df()
result

178. Ранговые баллы  
Напишите решение для определения ранга оценок. Ранжирование должно рассчитываться в соответствии со следующими правилами:  
- Оценки следует располагать в порядке убывания.  
- Если между двумя результатами есть ничья, оба показателя должны иметь одинаковый рейтинг.  
- В случае ничьей следующий номер в рейтинге должен быть следующим целым числом. Другими словами, между рангами не должно быть разрывов.  
Возвращает таблицу результатов, отсортированную в scoreпорядке убывания.

In [4]:
data = [[1, 3.5], [2, 3.65], [3, 4.0], [4, 3.85], [5, 4.0], [6, 3.65]]
scores = pd.DataFrame(data, columns=['id', 'score']).astype({'id':'Int64', 'score':'Float64'})
scores

,id,score
0,1,3.5
1,2,3.65
2,3,4.0
3,4,3.85
4,5,4.0
5,6,3.65


In [7]:
query = """
select
    score,
    dense_rank() over (order by score desc) as rank
from 
scores
"""
result = duckdb.query(query).to_df()
result

,score,rank
0,4.00,1
1,4.00,1
2,3.85,2
3,3.65,3
4,3.65,3
5,3.50,4


In [11]:
scores['rank'] = scores['score'].rank(method='dense', ascending=False).astype('Int64')
scores.sort_values('rank')[['score', 'rank']]

,score,rank
2,4.0,1
4,4.0,1
3,3.85,2
1,3.65,3
5,3.65,3
0,3.5,4


180. Последовательные номера  
Найдите все числа, которые встречаются как минимум три раза подряд.

In [13]:
data = [[1, 1], [2, 1], [3, 1], [4, 2], [5, 1], [6, 2], [7, 2]]
logs = pd.DataFrame(data, columns=['id', 'num']).astype({'id':'Int64', 'num':'Int64'})
logs

,id,num
0,1,1
1,2,1
2,3,1
3,4,2
4,5,1
5,6,2
6,7,2


In [ ]:
# мое решение
numbers = logs['num']
if len(numbers) > 0:
    result = []
    a = numbers[0]
    b = 1
    for i in range(1, len(numbers)):
        c = numbers[i]
        if c == a:
            b = b + 1
        else:
            b = 1
        a = c
        print(b)
        if b >= 3:
            result.append(numbers[i])
            b = 1
else:
    result = []
result_df = pd.DataFrame(result, columns=['ConsecutiveNums'])
result_df[['ConsecutiveNums']].drop_duplicates()

In [ ]:
# улучшение моего решения через цикл
nums = logs['num'].tolist()
result = []
count = 1
for i in range(1, len(nums)):
    if nums[i] == nums[i-1]:
        count += 1
        if count == 3:
            result.append(nums[i])
    else:
        count = 1
pd.DataFrame({'ConsecutiveNums': result})

In [ ]:
# еще одно решение GPT, через pandas
result = logs.loc[
    (logs['num'] == logs['num'].shift(1)) &
    (logs['num'] == logs['num'].shift(-1)),
    'num'
].drop_duplicates()
pd.DataFrame({'ConsecutiveNums': result})

In [ ]:
query = """
with t1 as(
    select 
        num,
        lag(num, 1, 0) over (order by id) as prev_1,
        lag(num, 2, 0) over (order by id) as prev_2
    from logs)
select distinct num as ConsecutiveNums
from t1
where num = prev_1 and num = prev_2
"""
result = duckdb.query(query).to_df()
result

184. Напишите решение для поиска сотрудников с самой высокой заработной платой в каждом из отделов.

In [4]:
data = [[1, 'Joe', 70000, 1], [2, 'Jim', 90000, 1], [3, 'Henry', 80000, 2], [4, 'Sam', 60000, 2], [5, 'Max', 90000, 1]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'departmentId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'departmentId':'Int64'})
data = [[1, 'IT'], [2, 'Sales']]
department = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})

In [5]:
employee

,id,name,salary,departmentId
0,1,Joe,70000,1
1,2,Jim,90000,1
2,3,Henry,80000,2
3,4,Sam,60000,2
4,5,Max,90000,1


In [6]:
department

,id,name
0,1,IT
1,2,Sales


In [13]:
# мое решение 
employee['rank'] = employee.groupby('departmentId')['salary'].rank(ascending=False).astype('int64')
merged = employee.merge(department, how='left', left_on='departmentId', right_on='id')
merged[merged['rank'] == 1].rename(columns={
    'name_y': 'Department', 
    'name_x': 'Employee', 
    'salary': 'Salary'
    })[['Department', 'Employee', 'Salary']]

,Department,Employee,Salary
1,IT,Jim,90000
2,Sales,Henry,80000
4,IT,Max,90000


In [8]:
# решение с сайта
merged = employee.merge(department, left_on='departmentId', right_on='id', how='left')
highest_salary = merged.loc[merged.groupby('departmentId')['salary'].transform('max') == merged['salary']]
result = highest_salary[['name_x', 'salary', 'name_y']].rename(columns={ 
    'name_y': 'Department',  # department name column
    'name_x': 'Employee',    # employee name column
    'salary': 'Salary'       # salary column
})
result[['Department', 'Employee', 'Salary']]

,Department,Employee,Salary
1,IT,Jim,90000
2,Sales,Henry,80000
4,IT,Max,90000


In [10]:
query = """
with t1 as(
    select
        d.name as Department,
        e.name as Employee,
        e.salary as Salary,
        dense_rank() over(partition by departmentId order by salary desc) as salary_rank
    from employee e
    join department d
    on e.departmentId = d.id
)
select
    Department,
    Employee,
    Salary
from t1
where salary_rank = 1
"""
result = duckdb.query(query).to_df()
result

,Department,Employee,Salary
0,Sales,Henry,80000
1,IT,Jim,90000
2,IT,Max,90000


550. Напишите решение, которое выведет долю игроков, повторно вошедших в систему на следующий день после первого входа, округлив до двух знаков после запятой . Другими словами, вам нужно определить количество игроков, вошедших в систему на следующий день после первого входа, и разделить его на общее количество игроков.

In [2]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-03-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})
activity.head()

,player_id,device_id,event_date,games_played
0,1,2,2016-03-01,5
1,1,2,2016-03-02,6
2,2,3,2017-06-25,1
3,3,1,2016-03-02,0
4,3,4,2018-07-03,5


In [ ]:
query = """
with t as(
    select
        player_id,
        min(event_date) as min_date
    from activity
    group by player_id)
select
    round(count(distinct t1.player_id) * 1.0 / (select count(distinct player_id) from activity),2) as fraction
from t t1 join activity t2
    on t1.min_date + 1 = t2.event_date and t1.player_id = t2.player_id
"""
result = duckdb.query(query).to_df()
result

In [41]:
activity['min_date'] = activity.groupby('player_id', as_index=False).event_date.transform(min)
pd.DataFrame([round(activity[activity['event_date'] == activity['min_date'] + pd.Timedelta(days=1)].player_id.nunique() / activity.player_id.nunique(),2)], columns=['fraction'])

C:\Users\user\AppData\Local\Temp\ipykernel_1916\3943309026.py:1: FutureWarning: The provided callable <built-in function min> is currently using SeriesGroupBy.min. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "min" instead.
  activity['min_date'] = activity.groupby('player_id', as_index=False).event_date.transform(min)


,fraction
0,0.33


570. Менеджеры, имеющие не менее 5 непосредственных подчиненных. Напишите решение для поиска менеджеров, у которых в подчинении не менее пяти человек .

In [16]:
data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]
employee = pd.DataFrame(data, columns=['id', 'name', 'department', 'managerId']).astype({'id':'Int64', 'name':'object', 'department':'object', 'managerId':'Int64'})
employee

,id,name,department,managerId
0,101,John,A,<NA>
1,102,Dan,A,101
2,103,James,A,101
3,104,Amy,A,101
4,105,Anne,A,101
5,106,Ron,B,101


In [4]:
query = """
with t as(
    select
        managerId
    from employee
    group by managerId
    having count(managerId) >= 5)
select name
from t join employee e
    on t.managerId = e.id
"""
result = duckdb.query(query).to_df()
result

,name
0,John


In [15]:
employee['count_emp'] = employee.groupby('managerId', as_index=False).managerId.transform('count')
t1 = employee[employee['count_emp'] >= 5][['managerId']].drop_duplicates()
t1.merge(employee, how='inner', left_on='managerId', right_on='id')[['name']]

,name
0,John


In [17]:
managers = employee.groupby('managerId', as_index=False).agg(reporting=('id', 'count'),).query('5 <= reporting')['managerId']
employee[employee['id'].isin(managers)][['name']]

,name
0,John


585. Инвестиции в 2016 году. Напишите решение, которое выведет сумму всех общих инвестиционных ценностей за 2016 год tiv_2016для всех страхователей, которые:  
- имеют такое же tiv_2015значение, как у одного или нескольких других страхователей  
- не находятся в том же городе, что и другие страхователи (т. е. пары атрибутов (lat, lon) должны быть уникальными).

In [56]:
data = [[1, 10, 5, 10, 10], [2, 20, 20, 20, 20], [3, 10, 30, 20, 20], [4, 10, 40, 40, 40]]
insurance = pd.DataFrame(data, columns=['pid', 'tiv_2015', 'tiv_2016', 'lat', 'lon']).astype({'pid':'Int64', 'tiv_2015':'Float64', 'tiv_2016':'Float64', 'lat':'Float64', 'lon':'Float64'})
insurance

,pid,tiv_2015,tiv_2016,lat,lon
0,1,10.0,5.0,10.0,10.0
1,2,20.0,20.0,20.0,20.0
2,3,10.0,30.0,20.0,20.0
3,4,10.0,40.0,40.0,40.0


In [ ]:
# мое решение
insurance['duble_lat_lon'] = insurance.groupby(['lat', 'lon'], as_index=False).pid.transform('count')
insurance['duble_tiv_2015'] = insurance.groupby('tiv_2015', as_index=False).pid.transform('count')
insurance[(insurance['duble_lat_lon'] == 1) & (insurance['duble_tiv_2015'] > 1)][['tiv_2016']].sum().round(2).to_frame(name='tiv_2016')

,tiv_2016
tiv_2016,45.0


In [ ]:
# решение с сайта, с моей точки зрения - не очень очевидное
df = pd.merge(insurance[insurance.duplicated('tiv_2015', keep = False)], insurance.drop_duplicates(['lat','lon'], keep = False), on = 'pid').iloc[:, [0, 2]]
pd.DataFrame({'tiv_2016': [round(df['tiv_2016_x'].sum(), 2)]})

,tiv_2016
0,45.0


In [4]:
query = """
WITH CheckedRows AS (
    SELECT
        tiv_2016,
        COUNT(*) OVER (PARTITION BY tiv_2015) AS tiv_2015_count,
        COUNT(*) OVER (PARTITION BY lat, lon) AS loc_count
    FROM
        insurance
)
SELECT
    ROUND(SUM(tiv_2016)::numeric, 2) AS tiv_2016
FROM
    CheckedRows
WHERE
    tiv_2015_count > 1
    AND loc_count = 1
"""
duckdb.query(query).to_df()

,tiv_2016
0,45.0


In [7]:
query = """
select
round(SUM(tiv_2016)::numeric,2) AS tiv_2016
from insurance
where tiv_2015 in (select tiv_2015 from insurance group by tiv_2015 having count(tiv_2015)>1)
and (lat, lon) in (select lat, lon from insurance group by lat, lon having count(*)=1)
"""
duckdb.query(query).to_df()

,tiv_2016
0,45.0


602. Запросы на добавление в друзья II: у кого больше друзей. Напишите решение, которое поможет найти людей с наибольшим количеством друзей и само количество друзей. Тестовые примеры составлены таким образом, что только у одного человека может быть наибольшее количество друзей.

In [15]:
data = [[1, 2, '2016/06/03'], [1, 3, '2016/06/08'], [2, 3, '2016/06/08'], [3, 4, '2016/06/09']]
request_accepted = pd.DataFrame(data, columns=['requester_id', 'accepter_id', 'accept_date']).astype({'requester_id':'Int64', 'accepter_id':'Int64', 'accept_date':'datetime64[ns]'})
request_accepted

,requester_id,accepter_id,accept_date
0,1,2,2016-06-03
1,1,3,2016-06-08
2,2,3,2016-06-08
3,3,4,2016-06-09


In [17]:
request_accepted.merge(request_accepted, how='left', left_on='requester_id', right_on='accepter_id')

,requester_id_x,accepter_id_x,accept_date_x,requester_id_y,accepter_id_y,accept_date_y
0,1,2,2016-06-03,<NA>,<NA>,NaT
1,1,3,2016-06-08,<NA>,<NA>,NaT
2,2,3,2016-06-08,1,2,2016-06-03
3,3,4,2016-06-09,1,3,2016-06-08
4,3,4,2016-06-09,2,3,2016-06-08


In [ ]:
# если есть только один человек с максимальным количеством друзей
query = """
with t1 as
(
    SELECT requester_id as id FROM request_accepted
    UNION ALL
    SELECT accepter_id as id FROM request_accepted
)    
select 
    id,
    count(id) as num
from t1
group by id
order by num desc
limit 1
"""
duckdb.query(query).to_df()

,id,num
0,3,3


In [16]:
# если у нескольких людей есть максимальное количество друзей
query = """
with t1 as
(
    SELECT requester_id as id FROM request_accepted
    UNION ALL
    SELECT accepter_id as id FROM request_accepted
),   
t2 as
(
    select 
        id,
        count(id) over(partition by id) as count_friends
    from t1
)
select distinct 
    id, 
    count_friends as num
from t2
where count_friends = (select max(count_friends) from t2)
"""
duckdb.query(query).to_df()

,id,num
0,3,3


608. Узел дерева. Каждый узел дерева может быть одного из трёх типов:  
"Листовой": если узел является листовым.  
"Корневой": если узел является корневым.  
"Внутренний": Если узел не является ни листовым, ни корневым.  
Напишите решение, которое будет сообщать тип каждого узла в дереве.

In [2]:
data = [[1, None], [2, 1], [3, 1], [4, 2], [5, 2]]
tree = pd.DataFrame(data, columns=['id', 'p_id']).astype({'id':'Int64', 'p_id':'Int64'})
tree

,id,p_id
0,1,<NA>
1,2,1
2,3,1
3,4,2
4,5,2


In [3]:
def classify_node(row):
    if pd.isna(row['p_id']):
        return 'Root'
    elif row['id'] not in tree['p_id'].values:
        return 'Leaf'
    else:
        return 'Inner'
tree['type'] = tree.apply(classify_node, axis=1)
tree[['id', 'type']]

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


In [4]:
query = """
with t1 as(
    select
        p_id as id,
        count(id) as count_compound
    from tree
    group by p_id
)
select
    id,
    case 
    when p_id is null then 'Root'
    when id in (select id from t1) then 'Inner'
    else 'Leaf' 
    end
    as type
from tree
"""
duckdb.query(query).to_df()

,id,type
0,1,Root
1,2,Inner
2,3,Leaf
3,4,Leaf
4,5,Leaf


626. Поменяйтесь местами. Напишите программу, которая меняет местами идентификаторы каждых двух последовательных студентов. Если количество студентов нечетное, идентификатор последнего студента не меняется местами с остальными.

In [5]:
data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id':'Int64', 'student':'object'})
seat

,id,student
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames


In [6]:
def exchange(x):
    if x % 2 == 0: return x - 1
    elif x == len(seat): return x
    else: return x + 1
seat['id'] = seat['id'].apply(exchange)
seat.sort_values(by='id')

,id,student
1,1,Doris
0,2,Abbot
3,3,Green
2,4,Emerson
4,5,Jeames


In [7]:
query = """
with t1 as(
    select max(id) as id
    from seat
)
select
    case when id % 2 = 0 then id - 1
    when id = (select id from t1) then id
    else id + 1
    end as id,
    student
from seat
order by id
"""
duckdb.query(query).to_df()

,id,student
0,1,Abbot
1,2,Doris
2,3,Emerson
3,4,Green
4,5,Jeames


1045. Покупатели, которые приобрели все товары. пишите решение для получения идентификаторов клиентов из Customer таблицы, которые купили все продукты в Product таблице.

In [8]:
data = [[1, 5], [2, 6], [3, 5], [3, 6], [1, 6]]
customer = pd.DataFrame(data, columns=['customer_id', 'product_key']).astype({'customer_id':'Int64', 'product_key':'Int64'})
data = [[5], [6]]
product = pd.DataFrame(data, columns=['product_key']).astype({'product_key':'Int64'})
customer

,customer_id,product_key
0,1,5
1,2,6
2,3,5
3,3,6
4,1,6


In [9]:
data

[[5], [6]]

In [10]:
query = """
with t1 as(
    select count(distinct(product_key)) from product
)
select customer_id 
from customer
group by(customer_id)
having count(distinct(product_key)) >= (select * from t1)
"""
duckdb.query(query).to_df()

,customer_id
0,3
1,1


In [26]:
count_prod = len(product)
customer.groupby('customer_id', as_index=False).agg({'product_key': 'nunique'}).query('product_key >= @count_prod')[['customer_id']]

,customer_id
0,1
2,3


1070. Анализ продаж продукции III. Напишите решение, которое позволит найти все продажи, совершённые в первый год по каждому товару. Для каждого product_id определите самый ранний year в таблице Sales. Возврат всех записей о продажах этого товара за этот год.

In [27]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
sales

,sale_id,product_id,year,quantity,price
0,1,100,2008,10,5000
1,2,100,2009,12,5000
2,7,200,2011,15,9000


In [28]:
query = """
with t1 as(    
    select
        product_id,
        year as  first_year,
        quantity,
        price,
        dense_rank() over(partition by product_id order by year) as  rank_year
    from sales
)
select
    product_id,
    first_year,
    quantity,
    price
from t1
where rank_year = 1
"""
duckdb.query(query).to_df()

,product_id,first_year,quantity,price
0,200,2011,15,9000
1,100,2008,10,5000


In [33]:
sales['rank_year'] = sales.groupby('product_id')['year'].transform(lambda x: x.rank(method='dense'))
sales[sales['rank_year'] == 1].rename(columns={'year': 'first_year'})[['product_id', 'first_year', 'quantity', 'price']]

,product_id,first_year,quantity,price
0,100,2008,10,5000
2,200,2011,15,9000


1158. Анализ рынка I. Напишите решение для поиска для каждого пользователя, дату присоединения и количество заказов, сделанных им в качестве покупателя в 2019.

In [34]:
data = [[1, '2018-01-01', 'Lenovo'], [2, '2018-02-09', 'Samsung'], [3, '2018-01-19', 'LG'], [4, '2018-05-21', 'HP']]
users = pd.DataFrame(data, columns=['user_id', 'join_date', 'favorite_brand']).astype({'user_id':'Int64', 'join_date':'datetime64[ns]', 'favorite_brand':'object'})
data = [[1, '2019-08-01', 4, 1, 2], [2, '2018-08-02', 2, 1, 3], [3, '2019-08-03', 3, 2, 3], [4, '2018-08-04', 1, 4, 2], [5, '2018-08-04', 1, 3, 4], [6, '2019-08-05', 2, 2, 4]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'item_id', 'buyer_id', 'seller_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'item_id':'Int64', 'buyer_id':'Int64', 'seller_id':'Int64'})
data = [[1, 'Samsung'], [2, 'Lenovo'], [3, 'LG'], [4, 'HP']]
items = pd.DataFrame(data, columns=['item_id', 'item_brand']).astype({'item_id':'Int64', 'item_brand':'object'})
users

,user_id,join_date,favorite_brand
0,1,2018-01-01,Lenovo
1,2,2018-02-09,Samsung
2,3,2018-01-19,LG
3,4,2018-05-21,HP


In [35]:
orders

,order_id,order_date,item_id,buyer_id,seller_id
0,1,2019-08-01,4,1,2
1,2,2018-08-02,2,1,3
2,3,2019-08-03,3,2,3
3,4,2018-08-04,1,4,2
4,5,2018-08-04,1,3,4
5,6,2019-08-05,2,2,4


In [36]:
items

,item_id,item_brand
0,1,Samsung
1,2,Lenovo
2,3,LG
3,4,HP


In [ ]:
query = """
select
    user_id as buyer_id,
    join_date,
    sum(case when to_char(order_date, 'YYYY') = '2019' then 1 else 0 end) as orders_in_2019
from orders o
full outer join users u
on u.user_id = o.buyer_id
group by user_id, join_date
order by buyer_id
"""
duckdb.query(query).to_df()

In [48]:
orders_2019 = orders[orders['order_date'].dt.year == 2019].groupby('buyer_id').agg(orders_in_2019=('order_date', 'count'))
users.rename(columns={'user_id': 'buyer_id'}).merge(orders_2019, how='outer', on='buyer_id').fillna(0)[['buyer_id', 'join_date', 'orders_in_2019']]

,buyer_id,join_date,orders_in_2019
0,1,2018-01-01,1.0
1,2,2018-02-09,2.0
2,3,2018-01-19,0.0
3,4,2018-05-21,0.0


1164. Цена товара на указанную дату. Изначально все продукты имеют цену 10. Напишите решение для нахождения цен на все продукты на дату 2019-08-16.

In [3]:
data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'], [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]
products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})
products

,product_id,new_price,change_date
0,1,20,2019-08-14
1,2,50,2019-08-14
2,1,30,2019-08-15
3,1,35,2019-08-16
4,2,65,2019-08-17
5,3,20,2019-08-18


In [4]:
query = """
with t1 as(
    select
        product_id,
        new_price as price,
        row_number() over(partition by product_id order by change_date desc) as row_price
    from products
    where change_date <= '2019-08-16'
),
t2 as(
    select distinct product_id from products
    )
select 
    t2.product_id,
    coalesce(t1.price, 10) as price
from t2 left join t1 on t1.product_id = t2.product_id and t1.row_price = 1
"""
duckdb.query(query).to_df()

,product_id,price
0,2,50
1,3,10
2,1,35


In [ ]:
df = products[products.change_date <= '2019-08-16']
latest_dates = df.groupby('product_id')['change_date'].transform('max')
df[df.change_date == latest_dates].merge(products.product_id.drop_duplicates(), how = 'right').fillna(10).rename(columns = {'new_price': 'price'}).iloc[:,[0,1]]

,product_id,price
0,1,35
1,2,50
2,3,10


1174. Немедленная доставка продуктов питания II. Если желаемая дата доставки клиента совпадает с датой заказа, то заказ считается немедленным; в противном случае он считается запланированным. Первым заказом клиента считается заказ с самой ранней датой оформления. Гарантируется, что у клиента будет ровно один первый заказ. Напишите решение для нахождения процента немедленных заказов в первых заказах всех клиентов, округленного до двух знаков после запятой 

In [6]:
data = [[1, 1, '2019-08-01', '2019-08-02'], [2, 2, '2019-08-02', '2019-08-02'], [3, 1, '2019-08-11', '2019-08-12'], [4, 3, '2019-08-24', '2019-08-24'], [5, 3, '2019-08-21', '2019-08-22'], [6, 2, '2019-08-11', '2019-08-13'], [7, 4, '2019-08-09', '2019-08-09']]
delivery = pd.DataFrame(data, columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype({'delivery_id':'Int64', 'customer_id':'Int64', 'order_date':'datetime64[ns]', 'customer_pref_delivery_date':'datetime64[ns]'})
delivery

,delivery_id,customer_id,order_date,customer_pref_delivery_date
0,1,1,2019-08-01,2019-08-02
1,2,2,2019-08-02,2019-08-02
2,3,1,2019-08-11,2019-08-12
3,4,3,2019-08-24,2019-08-24
4,5,3,2019-08-21,2019-08-22
5,6,2,2019-08-11,2019-08-13
6,7,4,2019-08-09,2019-08-09


In [16]:
first_orders = delivery.loc[delivery.groupby('customer_id')['order_date'].idxmin()]
first_orders['immediate'] = first_orders['order_date'] == first_orders['customer_pref_delivery_date']
pd.DataFrame({'immediate_percentage': [round(first_orders['immediate'].mean() * 100,2)]})

,immediate_percentage
0,50.0


In [14]:
query = """
with t1 as(
    select
        *,
        row_number() over (partition by customer_id order by order_date) as rn_date
    from delivery
)
select round(sum(case when order_date = customer_pref_delivery_date then 1 else 0 end)/count(distinct customer_id)::numeric * 100,2) as immediate_percentage 
from t1
where rn_date = 1
"""
duckdb.query(query).to_df()

,immediate_percentage
0,50.0


1193. Ежемесячные отчёты I. Напишите SQL-запрос, чтобы для каждого месяца и страны узнать количество транзакций и их общую сумму, а также количество одобренных транзакций и их общую сумму.

In [2]:
data = [[121, 'US', 'approved', 1000, '2018-12-18'], [122, 'US', 'declined', 2000, '2018-12-19'], [123, 'US', 'approved', 2000, '2019-01-01'], [124, 'DE', 'approved', 2000, '2019-01-07']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype({'id':'Int64', 'country':'object', 'state':'object', 'amount':'Int64', 'trans_date':'datetime64[ns]'})
transactions

,id,country,state,amount,trans_date
0,121,US,approved,1000,2018-12-18
1,122,US,declined,2000,2018-12-19
2,123,US,approved,2000,2019-01-01
3,124,DE,approved,2000,2019-01-07


In [ ]:
query = """
select 
    to_char(trans_date::date, 'YYYY-MM') as month,
    country,
    count(id) as trans_count,
    sum(case when state = 'approved' then 1 else 0 end) as approved_count,
    sum(amount) as trans_total_amount,
    sum(case when state = 'approved' then amount else 0 end) as approved_total_amount
from transactions
"""
duckdb.query(query).to_df()

In [3]:
df = transactions
df['month'] = transactions['trans_date'].dt.strftime('%Y-%m')
df['state'] = np.where(df['state'] == 'approved', 1, 0)
#df['approved_amount'] = df['amount'] * df['state']
df.groupby(['month', 'country'], as_index=False, dropna=False).agg(
    trans_count=('id', 'count'),
    approved_count=('state', 'sum'),
    trans_total_amount=('amount', 'sum'),
    approved_total_amount=('amount', lambda s: (s * df['state']).sum())
)

,month,country,trans_count,approved_count,trans_total_amount,approved_total_amount
0,2018-12,US,2,1,3000,1000
1,2019-01,DE,1,1,2000,2000
2,2019-01,US,1,1,2000,2000


1204. Последний, кто поместился в автобусе. Люди стоят в очереди, чтобы сесть в автобус. Однако вес автобуса ограничен 1000килограммами, поэтому некоторые пассажиры не смогут в него поместиться. Напишите решение, которое поможет найти person_nameпоследнего человека, который может поместиться в автобусе, не превышая допустимый вес. Тестовые примеры составлены таким образом, что первый человек не превышает допустимый вес. Обратите внимание, что на каждом повороте в автобус может сесть только один человек.

In [4]:
data = [[5, 'Alice', 250, 1], [4, 'Bob', 175, 5], [3, 'Alex', 350, 2], [6, 'John Cena', 400, 3], [1, 'Winston', 500, 6], [2, 'Marie', 200, 4]]
queue = pd.DataFrame(data, columns=['person_id', 'person_name', 'weight', 'turn']).astype({'person_id':'Int64', 'person_name':'object', 'weight':'Int64', 'turn':'Int64'})
queue

,person_id,person_name,weight,turn
0,5,Alice,250,1
1,4,Bob,175,5
2,3,Alex,350,2
3,6,John Cena,400,3
4,1,Winston,500,6
5,2,Marie,200,4


In [5]:
query = """
with t1 as(
    select
        person_name,
        sum(weight) over(order by turn) as cumulative_weight
    from queue
),
t2 as(
    select max(cumulative_weight) as max
    from t1
    where cumulative_weight <= 1000
)
select person_name 
from t1
where cumulative_weight = (select max from t2)
"""
duckdb.query(query).to_df()

,person_name
0,John Cena


In [6]:
df = queue
df = df.sort_values('turn')
df['cumulative_weight'] = df['weight'].cumsum()
df = df[df['cumulative_weight']<= 1000]
df.loc[[df['cumulative_weight'].idxmax()], ['person_name']]

,person_name
3,John Cena


1321. Рост ресторанного бизнеса. Вы — владелец ресторана и хотите проанализировать возможность расширения (каждый день у вас будет хотя бы один посетитель). Вычислите скользящее среднее значение суммы, которую клиент заплатил за последние семь дней (то есть за текущий день и шесть дней до него). average_amount должно быть округлено до двух знаков после запятой. Верните таблицу результатов, упорядоченную по visited_on возрастанию.

In [7]:
data = [[1, 'Jhon', '2019-01-01', 100], [2, 'Daniel', '2019-01-02', 110], [3, 'Jade', '2019-01-03', 120], [4, 'Khaled', '2019-01-04', 130], [5, 'Winston', '2019-01-05', 110], [6, 'Elvis', '2019-01-06', 140], [7, 'Anna', '2019-01-07', 150], [8, 'Maria', '2019-01-08', 80], [9, 'Jaze', '2019-01-09', 110], [1, 'Jhon', '2019-01-10', 130], [3, 'Jade', '2019-01-10', 150]]
customer = pd.DataFrame(data, columns=['customer_id', 'name', 'visited_on', 'amount']).astype({'customer_id':'Int64', 'name':'object', 'visited_on':'datetime64[ns]', 'amount':'Int64'})
customer

,customer_id,name,visited_on,amount
0,1,Jhon,2019-01-01,100
1,2,Daniel,2019-01-02,110
2,3,Jade,2019-01-03,120
3,4,Khaled,2019-01-04,130
4,5,Winston,2019-01-05,110
5,6,Elvis,2019-01-06,140
6,7,Anna,2019-01-07,150
7,8,Maria,2019-01-08,80
8,9,Jaze,2019-01-09,110
9,1,Jhon,2019-01-10,130


In [8]:
query = """
with t1 as(
    select
        visited_on,
        sum(amount) as sum_amount
    from customer
    group by visited_on
)
select 
    visited_on,
    sum(sum_amount) over(order by visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW) as amount,
    round(avg(sum_amount) over(order by visited_on ROWS BETWEEN 6 PRECEDING AND CURRENT ROW), 2) as average_amount
from t1
offset 6
"""
duckdb.query(query).to_df()

,visited_on,amount,average_amount
0,2019-01-07,860.0,122.86
1,2019-01-08,840.0,120.00
2,2019-01-09,840.0,120.00
3,2019-01-10,1000.0,142.86


In [9]:
df = customer
df = df.groupby('visited_on', as_index=False).agg(sum_amount=('amount', 'sum'))
df['amount'] = df['sum_amount'].rolling(window=7, min_periods=7).sum()
df['average_amount'] = df['sum_amount'].rolling(window=7, min_periods=7).mean().round(2)
df.drop('sum_amount', axis=1).iloc[6:]

,visited_on,amount,average_amount
6,2019-01-07,860.0,122.86
7,2019-01-08,840.0,120.00
8,2019-01-09,840.0,120.00
9,2019-01-10,1000.0,142.86


1341. Рейтинг фильма.   
- Найдите имя пользователя, который поставил наибольшее количество оценок фильмам. В случае равенства верните имя пользователя, которое лексикографически меньше.  
- Найдите название фильма с самым высоким средним рейтингом в February 2020. В случае равенства верните название фильма, которое лексикографически меньше.

In [32]:
data = [[1, 'Avengers'], [2, 'Frozen 2'], [3, 'Joker']]
movies = pd.DataFrame(data, columns=['movie_id', 'title']).astype({'movie_id':'Int64', 'title':'object'})
data = [[1, 'Daniel'], [2, 'Monica'], [3, 'Maria'], [4, 'James']]
users = pd.DataFrame(data, columns=['user_id', 'name']).astype({'user_id':'Int64', 'name':'object'})
data = [[1, 1, 3, '2020-01-12'], [1, 2, 4, '2020-02-11'], [1, 3, 2, '2020-02-12'], [1, 4, 1, '2020-01-01'], [2, 1, 5, '2020-02-17'], [2, 2, 2, '2020-02-01'], [2, 3, 2, '2020-03-01'], [3, 1, 3, '2020-02-22'], [3, 2, 4, '2020-02-25']]
movie_rating = pd.DataFrame(data, columns=['movie_id', 'user_id', 'rating', 'created_at']).astype({'movie_id':'Int64', 'user_id':'Int64', 'rating':'Int64', 'created_at':'datetime64[ns]'})
movies

,movie_id,title
0,1,Avengers
1,2,Frozen 2
2,3,Joker


In [11]:
users

,user_id,name
0,1,Daniel
1,2,Monica
2,3,Maria
3,4,James


In [12]:
movie_rating

,movie_id,user_id,rating,created_at
0,1,1,3,2020-01-12
1,1,2,4,2020-02-11
2,1,3,2,2020-02-12
3,1,4,1,2020-01-01
4,2,1,5,2020-02-17
5,2,2,2,2020-02-01
6,2,3,2,2020-03-01
7,3,1,3,2020-02-22
8,3,2,4,2020-02-25


In [40]:
top_user = (
    movie_rating
    .merge(users, on='user_id')
    .groupby('name', as_index=False)
    .agg(count_rating=('rating', 'count'))
    .sort_values(by=['count_rating', 'name'], ascending=[False, True])
    .iloc[0:1]
    .rename(columns={'name': 'results'})
    [['results']]
)

top_movie = (
    movie_rating
    .query("created_at.dt.strftime('%Y-%m') == '2020-02'")
    .merge(movies, on='movie_id')
    .groupby('title', as_index=False)
    .agg(mean_rating=('rating', 'mean'))
    .sort_values(by=['mean_rating', 'title'], ascending=[False, True])
    .iloc[0:1]
    .rename(columns={'title': 'results'})
    [['results']]
)

pd.concat([top_user, top_movie], ignore_index=True)

,results
0,Daniel
1,Frozen 2


In [ ]:
query = """
(select name as results
from MovieRating join users using(user_id)
order by 
    count(rating) over(partition by name) desc,
    name asc
limit 1)
union all
(select title as results
from MovieRating join movies using(movie_id)
where to_char(created_at, 'YYYY-MM') = '2020-02'
order by
    avg(rating) over(partition by title) desc,
    title asc
limit 1)
"""
duckdb.query(query).to_df()

1393. Прирост/убыток капитала. Напишите решение для расчёта прибыли/убытка от прироста капитала по каждой акции. Прибыль/убыток от прироста капитала по акции — это общая прибыль или убыток после покупки и продажи акции один или несколько раз.

In [3]:
data = [['Leetcode', 'Buy', 1, 1000], ['Corona Masks', 'Buy', 2, 10], ['Leetcode', 'Sell', 5, 9000], ['Handbags', 'Buy', 17, 30000], ['Corona Masks', 'Sell', 3, 1010], ['Corona Masks', 'Buy', 4, 1000], ['Corona Masks', 'Sell', 5, 500], ['Corona Masks', 'Buy', 6, 1000], ['Handbags', 'Sell', 29, 7000], ['Corona Masks', 'Sell', 10, 10000]]
stocks = pd.DataFrame(data, columns=['stock_name', 'operation', 'operation_day', 'price']).astype({'stock_name':'object', 'operation':'object', 'operation_day':'Int64', 'price':'Int64'})
stocks

,stock_name,operation,operation_day,price
0,Leetcode,Buy,1,1000
1,Corona Masks,Buy,2,10
2,Leetcode,Sell,5,9000
3,Handbags,Buy,17,30000
4,Corona Masks,Sell,3,1010
5,Corona Masks,Buy,4,1000
6,Corona Masks,Sell,5,500
7,Corona Masks,Buy,6,1000
8,Handbags,Sell,29,7000
9,Corona Masks,Sell,10,10000


In [8]:
df = stocks
df['adjusted_price'] = df.apply(lambda x: x['price'] if x['operation'] == 'Sell' else -x['price'], axis=1)
df.groupby('stock_name', as_index=False).agg(capital_gain_loss = ('adjusted_price', 'sum'))

,stock_name,capital_gain_loss
0,Corona Masks,9500
1,Handbags,-23000
2,Leetcode,8000


In [7]:
query = """
select
    stock_name,
    sum(case when operation = 'Buy' then -1 * price else price end) as capital_gain_loss
from stocks
group by stock_name
"""
duckdb.query(query).to_df()

,stock_name,capital_gain_loss
0,Corona Masks,9500.0
1,Handbags,-23000.0
2,Leetcode,8000.0


1907. Категории заработной платы. Напишите решение для расчета количества банковских счетов для каждой категории заработной платы. Категории заработной платы:  
- "Low Salary": Все зарплаты строго меньше, чем $20000.
- "Average Salary": Все зарплаты в включительно диапазоне [$20000, $50000].
- "High Salary": Все зарплаты строго больше, чем $50000.  
Таблица результатов должна содержать все три категории. Если в категории нет учетных записей, верните 0.

In [3]:
data = [[3, 108939], [2, 12747], [8, 87709], [6, 91796]]
accounts = pd.DataFrame(data, columns=['account_id', 'income']).astype({'account_id':'Int64', 'income':'Int64'})
accounts

,account_id,income
0,3,108939
1,2,12747
2,8,87709
3,6,91796


In [5]:
query = """
SELECT 
    unnest(array['Low Salary', 'Average Salary', 'High Salary']) AS "category",
    unnest(array[
        SUM (CASE WHEN income < 20000 THEN 1 ELSE 0 END),
        SUM (CASE WHEN income BETWEEN 20000 AND 50000 THEN 1 ELSE 0 END), 
        SUM (CASE WHEN income > 50000 THEN 1 ELSE 0 END)
    ]) AS "accounts_count"
FROM accounts
"""
duckdb.query(query).to_df()

,category,accounts_count
0,Low Salary,1.0
1,Average Salary,0.0
2,High Salary,3.0


In [6]:
pd.DataFrame({
        'category': ['Low Salary', 'Average Salary', 'High Salary'],
        'accounts_count': [
            (accounts.income < 20000).sum(),
            ((accounts.income >= 20000) & (accounts.income <= 50000)).sum(),
            (accounts.income > 50000).sum(),
        ]
    })

,category,accounts_count
0,Low Salary,1
1,Average Salary,0
2,High Salary,3


1934. Коэффициент подтверждения. Коэффициент подтверждения пользователя — это количество 'confirmed' сообщений, деленное на общее количество запрошенных сообщений с подтверждением. Коэффициент подтверждения пользователя, который не запрашивал никаких сообщений с подтверждением, равен 0. Округлите коэффициент подтверждения до двух знаков после запятой.
Напишите решение, которое найдет коэффициент подтверждения для каждого пользователя.

In [7]:
data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-07-14 14:00:00', 'timeout'], [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 12:58:28', 'confirmed'], [7, '2021-06-14 13:59:27', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'], [2, '2021-02-28 23:59:59', 'timeout']]
confirmations = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]', 'action':'object'})
signups

,user_id,time_stamp
0,3,2020-03-21 10:16:13
1,7,2020-01-04 13:57:59
2,2,2020-07-29 23:09:44
3,6,2020-12-09 10:39:37


In [8]:
confirmations

,user_id,time_stamp,action
0,3,2021-01-06 03:30:46,timeout
1,3,2021-07-14 14:00:00,timeout
2,7,2021-06-12 11:57:29,confirmed
3,7,2021-06-13 12:58:28,confirmed
4,7,2021-06-14 13:59:27,confirmed
5,2,2021-01-22 00:00:00,confirmed
6,2,2021-02-28 23:59:59,timeout


In [9]:
query = """
with t1 as(
    select
        user_id,
        round(avg(case when action = 'confirmed' then 1 else 0 end),2) as confirmation_rate
    from confirmations
    group by user_id
)
select 
    s.user_id,
    coalesce(confirmation_rate, 0) as confirmation_rate
from signups s left join t1 using(user_id)
"""
duckdb.query(query).to_df()

,user_id,confirmation_rate
0,7,1.0
1,2,0.5
2,3,0.0
3,6,0.0


In [10]:
df = confirmations
df = df.groupby('user_id', as_index=False).agg(confirmation_rate=('action', lambda x:  (x == 'confirmed').mean().round(2)))
signups.merge(df, how='left').fillna(0)[['user_id', 'confirmation_rate']]

,user_id,confirmation_rate
0,3,0.0
1,7,1.0
2,2,0.5
3,6,0.0


3220. Нечетные и четные транзакции. Напишите программу, которая находит сумму для нечетных и четных транзакций за каждый день. Если для определенной даты нет ни нечетных, ни четных транзакций, выведите 0.
Верните таблицу результатов, упорядоченную по transaction_date в по возрастаниюпорядку.

In [13]:
data = [[1, 150, '2024-07-01'], [2, 200, '2024-07-01'], [3, 75, '2024-07-01'], [4, 300, '2024-07-02'], [5, 50, '2024-07-02'], [6, 120, '2024-07-03']]
transactions = pd.DataFrame(
    data,
    columns=["transaction_id", "amount", "transaction_date"]
)
transactions = transactions.astype({
    "transaction_id": "int",
    "amount": "int",
    "transaction_date": "datetime64[ns]",
})
transactions

,transaction_id,amount,transaction_date
0,1,150,2024-07-01
1,2,200,2024-07-01
2,3,75,2024-07-01
3,4,300,2024-07-02
4,5,50,2024-07-02
5,6,120,2024-07-03


In [14]:
query = """
select
    transaction_date,
    sum(case when amount%2>0 then amount else 0 end) as odd_sum,
    sum(case when amount%2=0 then amount else 0 end) as even_sum
from transactions
group by transaction_date
order by transaction_date
"""
duckdb.query(query).to_df()

,transaction_date,odd_sum,even_sum
0,2024-07-01,75.0,350.0
1,2024-07-02,0.0,350.0
2,2024-07-03,0.0,120.0


In [15]:
transactions.groupby('transaction_date', as_index=False).agg(odd_sum=('amount', lambda x: x[x % 2 > 0].sum()), even_sum=('amount', lambda x: x[x % 2 == 0].sum()))

,transaction_date,odd_sum,even_sum
0,2024-07-01,75,350
1,2024-07-02,0,350
2,2024-07-03,0,120


3421. Найдите студентов, которые добились прогресса. Напишите решение, чтобы найти учеников, показавших улучшение успеваемости. Ученик считается показавшим улучшение успеваемости, если он соответствует обоим следующим условиям:  
- Сдавали экзамены по одному и тому же предмету как минимум в две разные даты
- Их последний результат по этому предмету выше, чем первый результат  
Возвращает таблицу результатов, упорядоченную по student_id, subject в порядкевозрастания.

In [18]:
data = [[101, 'Math', 70, '2023-01-15'], [101, 'Math', 85, '2023-02-15'], [101, 'Physics', 65, '2023-01-15'], [101, 'Physics', 60, '2023-02-15'], [102, 'Math', 80, '2023-01-15'], [102, 'Math', 85, '2023-02-15'], [103, 'Math', 90, '2023-01-15'], [104, 'Physics', 75, '2023-01-15'], [104, 'Physics', 85, '2023-02-15']]
scores = pd.DataFrame(
    data,
    columns=['student_id', 'subject', 'score', 'exam_date']
)
scores = scores.astype({   
    
    "student_id": "int",
    "subject": "str",
    "score": "int",
    "exam_date": "datetime64[ns]"
})
scores

,student_id,subject,score,exam_date
0,101,Math,70,2023-01-15
1,101,Math,85,2023-02-15
2,101,Physics,65,2023-01-15
3,101,Physics,60,2023-02-15
4,102,Math,80,2023-01-15
5,102,Math,85,2023-02-15
6,103,Math,90,2023-01-15
7,104,Physics,75,2023-01-15
8,104,Physics,85,2023-02-15


In [19]:
query = """
with t1 as(
    select distinct
        student_id,
        subject,
        first_value(score) over(partition by student_id, subject order by exam_date rows between unbounded preceding and unbounded following)
        as first_score,
        last_value(score) over(partition by student_id, subject order by exam_date rows between unbounded preceding and unbounded following)
        as latest_score
    from scores
)
select * from t1
where latest_score > first_score
order by student_id, subject
"""
duckdb.query(query).to_df()

,student_id,subject,first_score,latest_score
0,101,Math,70,85
1,102,Math,80,85
2,104,Physics,75,85


In [20]:
df = scores
df_sorted = df.sort_values(['student_id', 'subject', 'exam_date'])
result = (
    df_sorted
    .groupby(['student_id', 'subject'])
    .agg(
        first_score=('score', 'first'),   # первое значение после сортировки
        latest_score=('score', 'last')    # последнее значение
    )
    .reset_index()
    )
result = result[result['latest_score'] > result['first_score']]
result.sort_values(['student_id', 'subject']).reset_index(drop=True)

,student_id,subject,first_score,latest_score
0,101,Math,70,85
1,102,Math,80,85
2,104,Physics,75,85


3475. Распознавание образцов ДНК. Биологи изучают основные закономерности в последовательностях ДНК. Напишите решение, позволяющее определить sample_id по следующим закономерностям:
- Последовательности, которые начинаются с ATG (распространенный стартовый кодон)
- Последовательности, которые заканчиваютсяTAA, TAG или TGA (стоп-кодоны)
- Последовательности, содержащие мотив ATAT (простой повторяющийся паттерн)
- Последовательности, содержащие как минимум 3 последовательные G (например, GGG или GGGG)  
Верните таблицу результатов, отсортированную по sample_id в порядке возрастания.

In [21]:
data = [[1, 'ATGCTAGCTAGCTAA', 'Human'], [2, 'GGGTCAATCATC', 'Human'], [3, 'ATATATCGTAGCTA', 'Human'], [4, 'ATGGGGTCATCATAA', 'Mouse'], [5, 'TCAGTCAGTCAG', 'Mouse'], [6, 'ATATCGCGCTAG', 'Zebrafish'], [7, 'CGTATGCGTCGTA', 'Zebrafish']]
samples = pd.DataFrame(
    data,
    columns=['sample_id', 'dna_sequence', 'species']
)
samples = samples.astype({
    'sample_id': 'int',
    'dna_sequence': 'string',
    'species': 'string'
})
samples

,sample_id,dna_sequence,species
0,1,ATGCTAGCTAGCTAA,Human
1,2,GGGTCAATCATC,Human
2,3,ATATATCGTAGCTA,Human
3,4,ATGGGGTCATCATAA,Mouse
4,5,TCAGTCAGTCAG,Mouse
5,6,ATATCGCGCTAG,Zebrafish
6,7,CGTATGCGTCGTA,Zebrafish


In [22]:
query = """
select
    sample_id,
    dna_sequence,
    species,
    case when dna_sequence like 'ATG%' then 1 else 0 end as has_start,
    case when right(dna_sequence, 3) in ('TAA', 'TAG', 'TGA') then 1 else 0 end as has_stop,
    case when dna_sequence like '%ATAT%' then 1 else 0 end as has_atat,
    case when dna_sequence like '%GGG%' then 1 else 0 end as has_ggg
from samples
"""
duckdb.query(query).to_df()

,sample_id,dna_sequence,species,has_start,has_stop,has_atat,has_ggg
0,1,ATGCTAGCTAGCTAA,Human,1,1,0,0
1,2,GGGTCAATCATC,Human,0,0,0,1
2,3,ATATATCGTAGCTA,Human,0,0,1,0
3,4,ATGGGGTCATCATAA,Mouse,1,1,0,1
4,5,TCAGTCAGTCAG,Mouse,0,0,0,0
5,6,ATATCGCGCTAG,Zebrafish,0,1,1,0
6,7,CGTATGCGTCGTA,Zebrafish,0,0,0,0


In [24]:
samples.assign(
        has_start = samples.dna_sequence.str.startswith('ATG'),
        has_stop  = samples.dna_sequence.str.endswith(('TAA', 'TAG', 'TGA')),
        has_atat  = samples.dna_sequence.str.contains('ATAT'),
        has_ggg   = samples.dna_sequence.str.contains('GGG'))\
        .sort_values('sample_id')

,sample_id,dna_sequence,species,has_start,has_stop,has_atat,has_ggg
0,1,ATGCTAGCTAGCTAA,Human,True,True,False,False
1,2,GGGTCAATCATC,Human,False,False,False,True
2,3,ATATATCGTAGCTA,Human,False,False,True,False
3,4,ATGGGGTCATCATAA,Mouse,True,True,False,True
4,5,TCAGTCAGTCAG,Mouse,False,False,False,False
5,6,ATATCGCGCTAG,Zebrafish,False,True,True,False
6,7,CGTATGCGTCGTA,Zebrafish,False,False,False,False


In [25]:
df = samples
df['has_start'] = df['dna_sequence'].apply(lambda x: 1 if x[0 : 3] == 'ATG' else 0)
df['has_stop'] = df['dna_sequence'].apply(lambda x: 1 if x[-3 : ] in ['TAA', 'TAG', 'TGA'] else 0)
df['has_atat'] = df['dna_sequence'].apply(lambda x: 1 if 'ATAT' in x else 0)
df['has_ggg'] = df['dna_sequence'].apply(lambda x: 1 if 'GGG' in x else 0)
df

,sample_id,dna_sequence,species,has_start,has_stop,has_atat,has_ggg
0,1,ATGCTAGCTAGCTAA,Human,1,1,0,0
1,2,GGGTCAATCATC,Human,0,0,0,1
2,3,ATATATCGTAGCTA,Human,0,0,1,0
3,4,ATGGGGTCATCATAA,Mouse,1,1,0,1
4,5,TCAGTCAGTCAG,Mouse,0,0,0,0
5,6,ATATCGCGCTAG,Zebrafish,0,1,1,0
6,7,CGTATGCGTCGTA,Zebrafish,0,0,0,0


3497. Анализ конверсии подписок. Сервис подписки хочет анализировать модели поведения пользователей. Компания предлагает 7-дневную бесплатную пробную версию, после которой пользователи могут перейти на платный тариф или отменить подписку. Напишите решение, которое:
- Найдите пользователей, которые перешли с бесплатной пробной версии на платную подписку
- Рассчитайте среднюю продолжительность ежедневной активности каждого пользователя в течение бесплатного пробного периода (с округлением до 2 знаков после запятой)
- Рассчитайте среднюю продолжительность ежедневной активности каждого пользователя в течение платного периода подписки (с округлением до 2 знаков после запятой)  
Верните таблицу результатов, отсортированную по user_id в порядке возрастания.

In [26]:
data = [[1, '2023-01-01', 'free_trial', 45], [1, '2023-01-02', 'free_trial', 30], [1, '2023-01-05', 'free_trial', 60], [1, '2023-01-10', 'paid', 75], [1, '2023-01-12', 'paid', 90], [1, '2023-01-15', 'paid', 65], [2, '2023-02-01', 'free_trial', 55], [2, '2023-02-03', 'free_trial', 25], [2, '2023-02-07', 'free_trial', 50], [2, '2023-02-10', 'cancelled', 0], [3, '2023-03-05', 'free_trial', 70], [3, '2023-03-06', 'free_trial', 60], [3, '2023-03-08', 'free_trial', 80], [3, '2023-03-12', 'paid', 50], [3, '2023-03-15', 'paid', 55], [3, '2023-03-20', 'paid', 85], [4, '2023-04-01', 'free_trial', 40], [4, '2023-04-03', 'free_trial', 35], [4, '2023-04-05', 'paid', 45], [4, '2023-04-07', 'cancelled', 0]]
user_activity = pd.DataFrame(
    data,
    columns=['user_id', 'activity_date', 'activity_type', 'activity_duration']
)
user_activity = user_activity.astype({
    'user_id': 'int',
    'activity_date': 'datetime64[ns]',
    'activity_type':'str',
    'activity_duration': 'int'
})
user_activity

,user_id,activity_date,activity_type,activity_duration
0,1,2023-01-01,free_trial,45
1,1,2023-01-02,free_trial,30
2,1,2023-01-05,free_trial,60
3,1,2023-01-10,paid,75
4,1,2023-01-12,paid,90
5,1,2023-01-15,paid,65
6,2,2023-02-01,free_trial,55
7,2,2023-02-03,free_trial,25
8,2,2023-02-07,free_trial,50
9,2,2023-02-10,cancelled,0


In [28]:
query = """
select
    user_id,
    round(avg(case when activity_type = 'free_trial' then activity_duration else null end),2) as trial_avg_duration,
    round(avg(case when activity_type = 'paid' then activity_duration else null end),2) as paid_avg_duration
from user_activity
group by user_id
having round(avg(case when activity_type = 'paid' then activity_duration else null end),2) is not null
order by user_id
"""
duckdb.query(query).to_df()

,user_id,trial_avg_duration,paid_avg_duration
0,1,45.0,76.67
1,3,70.0,63.33
2,4,37.5,45.00


In [30]:
round2 = lambda x: round(x + 0.0001, 2)
df = user_activity
df['trial_duration'] = df[df['activity_type'] == 'free_trial']['activity_duration']
df['paid_duration'] = df[df['activity_type'] == 'paid']['activity_duration']
df.groupby('user_id', as_index=False).agg(
    trial_avg_duration = ('trial_duration', 'mean'),
    paid_avg_duration = ('paid_duration', 'mean')
).apply(round2).dropna().sort_values('user_id')

,user_id,trial_avg_duration,paid_avg_duration
0,1.0,45.0,76.67
2,3.0,70.0,63.33
3,4.0,37.5,45.00


3521. Найдите пары товаров, которые могут вам подойти. Amazon хочет внедрить функцию «Клиенты, купившие это, также купили...» на основе моделей совместных покупок. Напишите решение для:
- Определите отдельные пары товаров, которые часто покупаются вместе одними и теми же клиентами (где product1_id < product2_id)
- Для каждой пары товаров определите, сколько клиентов приобрели оба товара  
Пара товаров рассматривается для рекомендации если хотя бы 3 разные покупатели приобрели оба товара.
Верните таблицу результатов, упорядоченную по количеству клиентов в убывающем порядке, а в случае равенства — по product1_id в возрастающем порядке, а затем по product2_id в возрастающем порядке.

In [31]:
data = [[1, 101, 2], [1, 102, 1], [1, 103, 3], [2, 101, 1], [2, 102, 5], [2, 104, 1], [3, 101, 2], [3, 103, 1], [3, 105, 4], [4, 101, 1], [4, 102, 1], [4, 103, 2], [4, 104, 3], [5, 102, 2], [5, 104, 1]]
product_purchases = pd.DataFrame(
    data,
    columns=['user_id', 'product_id', 'quantity']
)
product_purchases = product_purchases.astype({
    "user_id": 'int64',
    "product_id": 'int64',
    "quantity": 'int64'
})
product_purchases

,user_id,product_id,quantity
0,1,101,2
1,1,102,1
2,1,103,3
3,2,101,1
4,2,102,5
5,2,104,1
6,3,101,2
7,3,103,1
8,3,105,4
9,4,101,1


In [32]:
data = [[101, 'Electronics', 100], [102, 'Books', 20], [103, 'Clothing', 35], [104, 'Kitchen', 50], [105, 'Sports', 75]]
product_info= pd.DataFrame(
    data,
    columns=['product_id', 'category', 'price']
)
product_info = product_info.astype({
    "product_id": 'int64',
    "category": 'string',
    "price": 'float64'
})
product_info

,product_id,category,price
0,101,Electronics,100.0
1,102,Books,20.0
2,103,Clothing,35.0
3,104,Kitchen,50.0
4,105,Sports,75.0


In [34]:
query = """
with t3 as(
    select
        t1.user_id,
        t1.product_id as product1_id,
        t2.product_id as product2_id
    from product_purchases as t1 join product_purchases as t2
        on t1.user_id = t2.user_id 
        and t2.product_id > t1.product_id
),
t4 as(
    select 
        product1_id, 
        product2_id,
        count(user_id) as customer_count
    from t3
    group by product1_id, product2_id
    having count(user_id) >= 3
)
select 
    product1_id, 
    product2_id,
    p1.category as product1_category,
    p2.category as product2_category,
    customer_count
from t4 join product_info p1
    on t4.product1_id = p1.product_id
    join product_info p2
        on t4.product2_id = p2.product_id
order by customer_count desc, product1_id, product2_id
"""
duckdb.query(query).to_df()

,product1_id,product2_id,product1_category,product2_category,customer_count
0,101,102,Electronics,Books,3
1,101,103,Electronics,Clothing,3
2,102,104,Books,Kitchen,3


In [35]:
df = pd.merge(product_purchases, product_purchases, on = 'user_id')
df = df.rename(columns = {'product_id_x': 'product1_id',
                            'product_id_y': 'product2_id'})
df = df[df.product1_id < df.product2_id]
df = df.groupby(['product1_id','product2_id'])['user_id'].nunique().reset_index()
df = df[df.user_id >= 3]
df = df.merge(product_info,  left_on = 'product1_id', 
                            right_on = 'product_id' , how = 'left')
df = df.merge(product_info,  left_on = 'product2_id', 
                            right_on = 'product_id' , how = 'left')
df = df.iloc[:,[0,1,4,7,2]]
df.columns  = ['product1_id', 'product2_id', 
                'product1_category', 'product2_category', 'customer_count']
df.sort_values(['customer_count', 'product1_id','product2_id'], ascending = [0,1,1])

,product1_id,product2_id,product1_category,product2_category,customer_count
0,101,102,Electronics,Books,3
1,101,103,Electronics,Clothing,3
2,102,104,Books,Kitchen,3


3564. Анализ сезонных продаж. Напишите программу, которая будет находить самую популярную категорию товаров для каждого сезона. Сезоны определяются следующим образом:
- Зима: декабрь, январь, февраль
- Весна: март, апрель, май
- Лето: июнь, июль, август
- Осень: сентябрь, октябрь, ноябрь    
Популярностькатегории определяется общим количеством проданных товаров в этом сезоне. Если есть равенство по количеству, выберите категорию с наибольшим общим доходом (quantity × price). Если по-прежнему наблюдается равенство, выберите категорию с наименьшим лексикографическим порядком.
Верните таблицу результатов, упорядоченную по сезонам в по возрастанию порядке.

In [37]:
data = [[1, 1, '2023-01-15', 5, 10.0], [2, 2, '2023-01-20', 4, 15.0], [3, 3, '2023-03-10', 3, 18.0], [4, 4, '2023-04-05', 1, 20.0], [5, 1, '2023-05-20', 2, 10.0], [6, 2, '2023-06-12', 4, 15.0], [7, 5, '2023-06-15', 5, 12.0], [8, 3, '2023-07-24', 2, 18.0], [9, 4, '2023-08-01', 5, 20.0], [10, 5, '2023-09-03', 3, 12.0], [11, 1, '2023-09-25', 6, 10.0], [12, 2, '2023-11-10', 4, 15.0], [13, 3, '2023-12-05', 6, 18.0], [14, 4, '2023-12-22', 3, 20.0], [15, 5, '2024-02-14', 2, 12.0]]
sales = pd.DataFrame(
    data,
    columns=['sale_id', 'product_id', 'sale_date', 'quantity', 'price'])
sales = sales.astype({
    'sale_id': 'int64', 
    'product_id': 'int64', 
    'sale_date': 'datetime64[ns]', 
    'quantity': 'int64', 
    'price': 'float64'})
sales

,sale_id,product_id,sale_date,quantity,price
0,1,1,2023-01-15,5,10.0
1,2,2,2023-01-20,4,15.0
2,3,3,2023-03-10,3,18.0
3,4,4,2023-04-05,1,20.0
4,5,1,2023-05-20,2,10.0
5,6,2,2023-06-12,4,15.0
6,7,5,2023-06-15,5,12.0
7,8,3,2023-07-24,2,18.0
8,9,4,2023-08-01,5,20.0
9,10,5,2023-09-03,3,12.0


In [39]:
data = [[1, 'Warm Jacket', 'Apparel'], [2, 'Designer Jeans', 'Apparel'], [3, 'Cutting Board', 'Kitchen'], [4, 'Smart Speaker', 'Tech'], [5, 'Yoga Mat', 'Fitness']]
products = pd.DataFrame(
    data,
    columns=['product_id', 'product_name', 'category'])
products = products.astype({'product_id': 'int64', 'product_name': 'string', 'category': 'string'})
products

,product_id,product_name,category
0,1,Warm Jacket,Apparel
1,2,Designer Jeans,Apparel
2,3,Cutting Board,Kitchen
3,4,Smart Speaker,Tech
4,5,Yoga Mat,Fitness


In [40]:
query = """
with t1 as(
    select *,
        CASE 
            WHEN EXTRACT(MONTH FROM sale_date) IN (12, 1, 2) THEN 'Winter'
            WHEN EXTRACT(MONTH FROM sale_date) IN (3, 4, 5)  THEN 'Spring'
            WHEN EXTRACT(MONTH FROM sale_date) IN (6, 7, 8)  THEN 'Summer'
            WHEN EXTRACT(MONTH FROM sale_date) IN (9, 10, 11) THEN 'Fall'
        END as season
    from sales join products using(product_id)
),
t2 as(
    select 
        season,
        category,
        sum(quantity) as total_quantity,
        sum(quantity * price) as total_revenue,
        row_number() over(partition by season order by sum(quantity) desc, sum(quantity * price) desc, category) as row_season
    from t1
    group by season, category
)
select
    season,
    category,
    total_quantity,
    total_revenue
from t2
where row_season = 1
"""
duckdb.query(query).to_df()

,season,category,total_quantity,total_revenue
0,Spring,Kitchen,3.0,54.0
1,Fall,Apparel,10.0,120.0
2,Winter,Apparel,9.0,110.0
3,Summer,Tech,5.0,100.0


In [42]:

df = sales
df['season'] = df['sale_date'].apply(
    lambda x: 'Winter' if x.month in [12, 1, 2]
            else 'Spring' if x.month in [3, 4, 5]
            else 'Summer' if x.month in [6, 7, 8]
            else 'Fall'
)
df['revenue'] = df['quantity'] * df['price']
df = df.merge(products)
df = df.groupby(['season', 'category'], as_index=False).agg(
    total_quantity = ('quantity', 'sum'),
    total_revenue = ('revenue', 'sum')
)
df = df.sort_values(
    ['season', 'total_quantity', 'total_revenue', 'category'],
    ascending = [True, False, False, True]
)
df['row_season'] = df.groupby('season').cumcount() + 1
df[df['row_season'] == 1][
    ['season', 'category', 'total_quantity', 'total_revenue']
].reset_index(drop=True)

,season,category,total_quantity,total_revenue
0,Fall,Apparel,10,120.0
1,Spring,Kitchen,3,54.0
2,Summer,Tech,5,100.0
3,Winter,Apparel,9,110.0


3580. Как найти сотрудников, которые постоянно совершенствуются. Напишите решение для поиска сотрудников, которые стабильно повышали свою эффективность на протяжении последних трех аттестаций.
- Чтобы сотрудник мог быть рассмотрен, у него должно быть не менее 3 отзывов
- Последние 3 отзывы сотрудника должны показывать строго возрастающие оценки (каждый отзыв лучше предыдущего)
- Используйте самые последние 3 отзывы, основанные на review_date для каждого сотрудника
- Рассчитайте показатель улучшения как разницу между последней оценкой и самой ранней оценкой из последних 3 отзывов  
Верните таблицу результатов, упорядоченную по баллам улучшения в убывающем порядке, затем по названию в возрастающем порядке.

In [2]:
data = [[1, 'Alice Johnson'], [2, 'Bob Smith'], [3, 'Carol Davis'], [4, 'David Wilson'], [5, 'Emma Brown']]
employees = pd.DataFrame(
    data,
    columns=['employee_id', 'name'])
employees = employees.astype({
    'employee_id': 'int',
    'name': 'str'})
employees

,employee_id,name
0,1,Alice Johnson
1,2,Bob Smith
2,3,Carol Davis
3,4,David Wilson
4,5,Emma Brown


In [3]:
data = [[1, 1, '2023-01-15', 2], [2, 1, '2023-04-15', 3], [3, 1, '2023-07-15', 4], [4, 1, '2023-10-15', 5], [5, 2, '2023-02-01', 3], [6, 2, '2023-05-01', 2], [7, 2, '2023-08-01', 4], [8, 2, '2023-11-01', 5], [9, 3, '2023-03-10', 1], [10, 3, '2023-06-10', 2], [11, 3, '2023-09-10', 3], [12, 3, '2023-12-10', 4], [13, 4, '2023-01-20', 4], [14, 4, '2023-04-20', 4], [15, 4, '2023-07-20', 4], [16, 5, '2023-02-15', 3], [17, 5, '2023-05-15', 2]]
performance_reviews = pd.DataFrame(
    data,
    columns=['review_id', 'employee_id', 'review_date', 'rating'])
performance_reviews = performance_reviews.astype({
    'review_id': 'int',
    'employee_id': 'int',
    'review_date': 'datetime64[ns]',
    'rating': 'float' })
performance_reviews

,review_id,employee_id,review_date,rating
0,1,1,2023-01-15,2.0
1,2,1,2023-04-15,3.0
2,3,1,2023-07-15,4.0
3,4,1,2023-10-15,5.0
4,5,2,2023-02-01,3.0
5,6,2,2023-05-01,2.0
6,7,2,2023-08-01,4.0
7,8,2,2023-11-01,5.0
8,9,3,2023-03-10,1.0
9,10,3,2023-06-10,2.0


In [4]:
query = """
WITH ranked AS (
    SELECT
        employee_id,
        rating,
        DENSE_RANK() OVER (PARTITION BY employee_id ORDER BY review_date DESC) AS rn
    FROM performance_reviews
),
last_3 AS (
    SELECT
        employee_id,
        MAX(CASE WHEN rn = 1 THEN rating END) AS latest_rating,
        MAX(CASE WHEN rn = 2 THEN rating END) AS prev_rating,
        MAX(CASE WHEN rn = 3 THEN rating END) AS oldest_rating
    FROM ranked
    WHERE rn <= 3
    GROUP BY employee_id
    HAVING COUNT(*) = 3  -- ровно 3 оценки
),
filtered AS (
    SELECT *
    FROM last_3
    WHERE prev_rating > oldest_rating
      AND latest_rating > prev_rating
)
SELECT
    f.employee_id,
    e.name,
    (f.latest_rating - f.oldest_rating) AS improvement_score
FROM filtered f
JOIN employees e ON f.employee_id = e.employee_id
ORDER BY improvement_score DESC, name;
"""
duckdb.query(query).to_df()

,employee_id,name,improvement_score
0,2,Bob Smith,3.0
1,1,Alice Johnson,2.0
2,3,Carol Davis,2.0


In [5]:
df = performance_reviews
df = df.sort_values(['employee_id', 'review_date'], ascending=[True, False])
df['rank'] = df.groupby('employee_id').cumcount() + 1
df = df[df['rank'] <= 3]
valid_employees = df[df['rank'] >=3]['employee_id'].unique()
df = df[df['employee_id'].isin(valid_employees)]
df = df.pivot(
    index = 'employee_id',
    columns = 'rank',
    values = 'rating'
).reset_index()
df = df[(df[1] > df[2]) & (df[2] > df[3])]
df['improvement_score'] = df[1] - df[3]
df = df.merge(employees)
df[['employee_id', 'name', 'improvement_score']].sort_values(['improvement_score', 'name'], ascending=[False, True])

,employee_id,name,improvement_score
1,2,Bob Smith,3.0
0,1,Alice Johnson,2.0
2,3,Carol Davis,2.0
